NOTES

1. Serve the model using vLLM like so - vllm serve Qwen/Qwen3-30B-A3B-Instruct-2507 --tensor-parallel-size 4 --port 8000 


2. Serve sglang model and then modify _openai_base.py like so 
add this in the for chunk in chunks portion after -
choice = chunk.choices[0]
<br>
delta = choice.delta
<br>
rc = getattr(delta, "reasoning_content", None)
if rc:
    reasoning_buf += rc


3. Then after that for loop and before this statement 
(usage.total_latency_ms += (time.time() - _t0) * 1000 
<br>
self.state.add_usage(usage)) 
<br>    
add -
<br>
if reasoning_buf:
    yield self.state.apply_capture(
        name="reasoning",
        value=reasoning_buf,
        log_prob=1,      # unknown; best-effort default
        is_append=False,
    )

In [5]:
import os
from pydantic import BaseModel
import guidance

In [6]:
# from pydantic import BaseModel, Field, conint
# from typing import Literal

# class Person(BaseModel):
#     name: str
#     age: conint(ge=0, le=150)
#     country: Literal["UK", "IN"]

# schema: dict = Person.model_json_schema()   # Pydantic v2
# # schema_v1: dict = Person.schema()        # Pydantic v1
# schema

In [7]:
# # vllm hosted model using LiteLLM

# litellm_desc = {
#     "model_name": "Qwen/Qwen3-30B-A3B-Instruct-2507",
#     "litellm_params": {  # params for litellm completion/embedding call
#         "model": "hosted_vllm/Qwen/Qwen3-30B-A3B-Instruct-2507",
#         "api_key": os.environ.get("VLLM_API_KEY", "NO_KEY"), # set your vLLM API key if needed
#         "api_base": "http://localhost:8000/v1", # change to your vLLM API base URL
#     },
# }
# base_lm = guidance.models.experimental.LiteLLM(model_description=litellm_desc, echo=True)

In [8]:
# from openai import OpenAI
# client = OpenAI(base_url="http://127.0.0.1:30000/v1", api_key="None")
# r = client.chat.completions.create(
#     model="Qwen/Qwen3-30B-A3B-Thinking-2507",
#     messages=[{"role":"user","content":"1+3?"}],
#     # extra_body={"separate_reasoning": True},  # key bit
# )
# print("REASONING:", r.choices[0].message.reasoning_content)
# print("ANSWER:", r.choices[0].message.content)


In [5]:
# using sglang hosted model
# need to add the experimental sglang from nightly version of guidance - https://github.com/guidance-ai/guidance/blob/main/guidance/models/experimental/_sglang.py

base_lm = guidance.models.experimental.SglangModel(model="Qwen/Qwen3-30B-A3B-Thinking-2507", base_url="http://127.0.0.1:40000/v1" ,echo=True, api_key = os.environ.get("OPENAI_API_KEY", "NO_KEY"))

In [ ]:
# quick test
# @guidance
# def run_gen_test(lm):
#     with guidance.user():
#         lm += "What is the capital of France? and its population?"
#         lm += "Format your answer as follows: Capital: <capital>, Population: <population>"

#     with guidance.assistant():
#         lm += guidance.gen(max_tokens=1024, temperature=0.7, name="answer")
#         print("Reasoning:\n", lm.get("reasoning", ""))
#         print(lm["answer"])
#     return lm

# lm = base_lm + run_gen_test()

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

Reasoning:
 Hmm, the user is asking for the capital of France and its population, with a specific format requirement. They want it as "Capital: <capital>, Population: <population>". 

I recall that Paris is the capital of France, that's basic geography. For population, I need to be precise. The latest reliable data from sources like the French National Institute of Statistics shows Paris city proper has about 2.1 million people, but the metropolitan area is much larger. The user didn't specify which population figure they want, so I should clarify.

The question says "its population" referring to the capital city, not the whole country. So I'll go with Paris city limits. The most recent official figure I know is 2,140,526 as of 2021. But since the user might want the most current estimate, I'll use the commonly cited approximate figure of 2.1 million to keep it clean. 

I notice the user is very specific about the format - they're probably copying this into some system that requires ex

In [ ]:
# lm.__dict__

{'echo': True,
 '_interpreter': <guidance.models.experimental._sglang.SglangInterpreter at 0x153e86668980>,
 '_active_blocks': {<guidance.library._block.Block at 0x153e866687a0>: 154},
 'sampling_params': {},
 '_parent': <guidance.models.experimental._sglang.SglangModel at 0x153e86b84da0>,
 '_parent_id': 19,
 '_id': 20,
 '_trace_nodes': {10:[]:[TokenOutput:':':True:13.835430145263672:Token(token=':', bytes=b':', prob=1.0, masked=False):[Token(token=':', bytes=b':', prob=1.0, masked=False), Token(token='：', bytes=b'\xef\xbc\x9a', prob=1.813248599208411e-16, masked=False), Token(token=' ', bytes=b' ', prob=1.5167425099519682e-16, masked=False), Token(token='.', bytes=b'.', prob=7.969440121294002e-18, masked=False), Token(token=':\n', bytes=b':\n', prob=2.496819144761755e-18, masked=False)]],
  11:[]:[TokenOutput:' ':True:5.9604644775390625:Token(token=' ', bytes=b' ', prob=1.0, masked=False):[Token(token=' ', bytes=b' ', prob=1.0, masked=False), Token(token='2', bytes=b'2', prob=1.081194

In [9]:
import torch
torch.cuda.empty_cache()

import time
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
from pydantic import BaseModel, Field, RootModel
from typing import Optional, Union, Literal, ForwardRef, List, Any
from enum import Enum
from guidance import models, system, user, assistant, json as gen_json, gen
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from pydantic import BaseModel, Field, RootModel
from typing import Optional, Union
from enum import Enum
from guidance import models, system, user, assistant, json as gen_json
import guidance
from utils import timing_decorator

In [15]:
from pydantic import BaseModel, Field
from typing import List, Any

class DomainVariable(BaseModel):
    """Defines a single variable the LLM can use in the computation graph."""
    name: str = Field(..., description="The unique identifier for the variable.")
    description: str = Field(..., description="A detailed explanation of what this variable represents.")
    # Optional: You could add type hints, units, etc. for more advanced validation
    unit: Optional[str] = Field(..., description="The unit of the variable")
    data_type : type = Field(..., description="The data type of the variable")


In [17]:
def create_dynamic_variable_enum(charge_category: str, charge_category_variables: dict) -> type(Enum):
    """
    Creates a new Enum class containing only the variables relevant
    to the specified charge category.
    """
    variable_names = charge_category_variables.get(charge_category)
    if not variable_names:
        raise ValueError(f"Unknown charge category: {charge_category}")
    
    # The dictionary for the Enum must have {MEMBER_NAME: value}
    # We'll use uppercase for the member name for convention.
    enum_dict = {name.upper(): name for name in variable_names}
    
    # Create the Enum class dynamically
    return Enum("Var", enum_dict)

# Var = create_dynamic_variable_enum("transfer_passenger_charge", CHARGE_CATEGORY_VARIABLES)
# print(Var.TRANSFER_PASSENGER_COUNT.value)

In [18]:
# allowed_variables = [el.value for el in list(Var)]
# allowed_variables

In [19]:
from enum import Enum

class Units(str, Enum):
    # General
    UNITLESS = "UNITLESS"
    PERCENT = "PERCENT"

    # Time
    NANOSECONDS = "NANOSECONDS"
    MICROSECONDS = "MICROSECONDS"
    MILLISECONDS = "MILLISECONDS"
    SECONDS = "SECONDS"
    MINUTES = "MINUTES"
    HOURS = "HOURS"
    DAYS = "DAYS"
    WEEKS = "WEEKS"

    # Length
    MILLIMETERS = "MILLIMETERS"
    CENTIMETERS = "CENTIMETERS"
    METERS = "METERS"
    KILOMETERS = "KILOMETERS"
    INCHES = "INCHES"
    FEET = "FEET"
    YARDS = "YARDS"
    MILES = "MILES"

    # Mass
    MILLIGRAMS = "MILLIGRAMS"
    GRAMS = "GRAMS"
    KILOGRAMS = "KILOGRAMS"
    TONNES = "TONNES"

    # Speed / Acceleration
    METERS_PER_SECOND = "METERS_PER_SECOND"
    KILOMETERS_PER_HOUR = "KILOMETERS_PER_HOUR"
    MILES_PER_HOUR = "MILES_PER_HOUR"
    METERS_PER_SECOND_SQUARED = "METERS_PER_SECOND_SQUARED"

    # Data size
    BYTES = "BYTES"
    KILOBYTES = "KILOBYTES"
    MEGABYTES = "MEGABYTES"
    GIGABYTES = "GIGABYTES"
    TERABYTES = "TERABYTES"
    BITS = "BITS"

    # Data rates
    BITS_PER_SECOND = "BITS_PER_SECOND"
    KILOBITS_PER_SECOND = "KILOBITS_PER_SECOND"
    MEGABITS_PER_SECOND = "MEGABITS_PER_SECOND"
    GIGABITS_PER_SECOND = "GIGABITS_PER_SECOND"
    BYTES_PER_SECOND = "BYTES_PER_SECOND"

    # Currencies (suggest using ISO codes)
    EUROS = "EUROS"
    USD = "USD"
    GBP = "GBP"
    JPY = "JPY"

    # ML / LLM specific
    TOKENS = "TOKENS"
    TOKENS_PER_SECOND = "TOKENS_PER_SECOND"
    SAMPLES = "SAMPLES"
    SAMPLES_PER_SECOND = "SAMPLES_PER_SECOND"
    REQUESTS_PER_SECOND = "REQUESTS_PER_SECOND"
    FLOPS = "FLOPS"
    GFLOPS = "GFLOPS"
    TFLOPS = "TFLOPS"

In [20]:
from enum import Enum

class MathOperator(str, Enum):
    ADD = "ADD"
    SUBTRACT = "SUBTRACT"
    MULTIPLY = "MULTIPLY"
    DIVIDE = "DIVIDE"
    FLOOR_DIVIDE = "FLOOR_DIVIDE"
    MOD = "MOD"
    POWER = "POWER"
    ROOT = "ROOT"
    SQRT = "SQRT"
    FLOOR = "FLOOR"
    CEILING = "CEILING"
    MIN = "MIN"
    MAX = "MAX"
    

class Comparator(str, Enum):
    EQUAL_TO = "EQUAL_TO"
    NOT_EQUAL_TO = "NOT_EQUAL_TO"
    LESS_THAN = "LESS_THAN"
    LESS_THAN_OR_EQUAL_TO = "LESS_THAN_OR_EQUAL_TO"
    GREATER_THAN = "GREATER_THAN"
    GREATER_THAN_OR_EQUAL_TO = "GREATER_THAN_OR_EQUAL_TO"
    

In [21]:
from pydantic import BaseModel, Field, RootModel
from typing import Union, Literal
from enum import Enum

# Separate enums for clarity and type safety
# class MathOperator(str, Enum):
#     ADD = "ADD"
#     MULTIPLY = "MULTIPLY"
#     DIVIDE = "DIVIDE"
#     CEIL = "CEIL"
    
# class Comparator(str, Enum):
#     GREATER_THAN = "GREATER_THAN"
#     LESS_THAN = "LESS_THAN"
#     EQUAL_TO = "EQUAL_TO"

# class Units(str, Enum):
#     HOURS = "HOURS"
#     MINUTES = "MINUTES"
#     EUROS = "EUROS"
#     PERCENT = "PERCENT"
#     UNITLESS = "UNITLESS"
    
# --- Node Definitions ---

class ValueNode(BaseModel):
    type: Literal["VALUE"] = "VALUE"
    value: float
    description: str
    unit: Units

class CategoricalValueNode(BaseModel):
    type: Literal["CATEGORICAL_VALUE"] = "CATEGORICAL_VALUE"
    value: str
    description: str

class VariableNode(BaseModel):
    type: Literal["VARIABLE"] = "VARIABLE"
    name: str 
    description: str
    unit: Units

class BinaryOpNode(BaseModel):
    """Node for mathematical operations that produce a number."""
    type: Literal["BINARY_OPERATION"] = "BINARY_OPERATION"
    operator: MathOperator
    left: 'AnyNode'
    right: 'AnyNode'

class ComparisonNode(BaseModel):
    """Node for comparison operations that produce a boolean."""
    type: Literal["COMPARISON"] = "COMPARISON"
    operator: Comparator
    left: 'AnyNode'
    right: 'AnyNode'

class ConditionalNode(BaseModel):
    """Node for if-then-else logic."""
    type: Literal["CONDITIONAL"] = "CONDITIONAL"
    condition: ComparisonNode # Condition must be a comparison
    if_true: 'AnyNode'
    if_false: 'AnyNode'

# --- Recursive Setup ---

AnyNode = Union[
    ValueNode,
    CategoricalValueNode, 
    VariableNode, 
    BinaryOpNode, 
    ConditionalNode
]

# Use model_rebuild() to safely resolve all forward references
BinaryOpNode.model_rebuild()
ConditionalNode.model_rebuild()
ComparisonNode.model_rebuild()

class Node(RootModel):
    root: BinaryOpNode

In [22]:
# from pydantic import BaseModel, Field, field_validator
# from typing import List, Dict, Any, Optional
# from enum import Enum


# class ParameterStatus(str, Enum):
#     """An enumeration for clear, explicit parameter statuses."""
#     KNOWN = "KNOWN"
#     SYMBOLIC = "SYMBOLIC"

# class VariableType(str, Enum):
#     """An enumeration for clear, explicit parameter statuses."""
#     CATEGORICAL = "CATEGORICAL"
#     NUMERIC = "NUMERIC"

# class ParameterDetail(BaseModel):
#     """A structured model to describe each parameter identified from the query."""
#     name: str = Field(..., description="The name of the parameter.")
#     status: ParameterStatus = Field(..., description="Whether the parameter's value is known from the query or is a symbolic variable.")
#     type_of_variable: VariableType = Field(..., description="whether the variable is numeric or categorical")
#     # value: Optional[Any] = Field(None, description="The actual value of the parameter, if its status is 'KNOWN'. Must be null if status is 'SYMBOLIC'.")
    


# class ReasoningSchemaStep1(BaseModel):
#     query_parameters: List[ParameterDetail] = Field(
#         ...,
#         description="A structured list of all parameters identified from the query and their status."
#     )
# class ReasoningSchemaStep2(BaseModel):
#     """
#     A simplified schema for Step 2 that captures all constants and rules as a simple list of descriptive strings.
#     """
#     identified_constants_and_rules: List[str] = Field(
#         ...,
#         description="A comprehensive list of all facts, constants, and conditional rules extracted from the document that are necessary for the final calculation. Each string in the list should be a self-contained, clear statement. For example: 'The rate for a Narrow Satellite stand is €32.90 per 15 minutes' or 'A 100% surcharge is applied if parking duration is between 48 and 72 hours'."
#     )
# class ReasoningSchemaStep3(BaseModel):
#     synthesis_plan: str = Field(
#         ...,
#         description="A concise, step-by-step plan describing how the variables and constants are combined into the final computation graph."
#     )
# class ReasoningSchemaStep4(BaseModel):
#     rethink: str = Field(
#         ...,
#         description="Final check to ensure the plan correctly uses variables and constants and handles all logic from the document."
#     )


In [23]:
@guidance
def create_graph_with_cot(llm, allowed_variables_prompt, thought1, document, query, output_schema):
    
    with system():
        llm += f"""You are an expert system that converts textual calculation rules into structured JSON expression trees.
        You MUST think step-by-step and reason before generating the final JSON.
        
        **Reasoning Guidelines:**
        1.  **Analyze Query Parameters:** Identify all relevant parameters from the user's query. For each parameter, create a structured object specifying its 'name', its 'status' ('KNOWN' if the value is given, or 'SYMBOLIC' if it's a variable), its 'type_of_variable' ('NUMERIC' if the value is a number or 'CATEGORICAL' if the value is categories), and its 'value' (or null if symbolic). For example: `[ {{"name": "aircraft_stand_type", "status": "KNOWN", "type_of_variable": "CATEGORICAL", "value": "Wide Remote"}}, {{"name": "parking_duration_hours", "status": "SYMBOLIC", "type_of_variable": "NUMERIC", "value": null}} ]`
        **Allowed Variables for this Task:**
        ---
        {allowed_variables_prompt}
        ---
        2. **Identify All Relevant Information**: Review the document and extract every fact, constant, and conditional rule needed for the calculation. Each piece of information should be written as a clear, self-contained sentence and collected into a list of strings.
        3.  **Synthesize Plan:** Briefly describe how you will combine these pieces into a final expression tree.
        4. **Rethink and Finalize Approach**: Before processing with generation, rethink your progress so far and make adjustments if necessary, then finalize and proceed to generate the expression tree.

        **Crucial Rule 1:** If a parameter from the 'Allowed Variables' list is given a specific value in the query, you MUST treat it as a fixed value to find constants. You MUST NOT include it as a `VARIABLE` node in the final JSON.
        **Crucial Rule 2**: If a calculation path or value depends on the value of a symbolic variable, you MUST capture the rules for all possible values and represent this logic using CONDITIONAL nodes in the final expression tree. You MUST NOT assume a default value for the variable to simplify the logic.
        **Crucial Rule 3 (Categorical Variables)**: Variables whose domain is categorical (e.g., string-valued like 'flight_type', 'stand_type') must be handled explicitly.
            - If the query supplies a category, treat it as fixed (per Rule 1).
            - If the query does not supply a category, mark the variable as 'SYMBOLIC' in Step 1 and ensure the final expression covers all category-dependent branches stated in the document using `CONDITIONAL` nodes. Do NOT assume a default category. Always use `CATEGORICAL_VALUE_NODE` for category constants.
            - Ensure every variable in the Allowed Variables list appears in Step 1 with a status of KNOWN or SYMBOLIC.
            - Encoding guidance: when checking a categorical branch, use a `COMPARISON` node with operator `EQUAL_TO`, `left` as a `VARIABLE` node for the categorical variable, and `right` as a `CATEGORICAL_VALUE_NODE` whose `value` is the category string (e.g., "International").

        **Crucial Rule 4 (Numeric vs Categorical Comparisons)**:
            - Numeric variables must be compared against numeric `VALUE` nodes whose `value` is a number.
            - Categorical variables must be compared against `CATEGORICAL_VALUE_NODE` nodes whose `value` is the category string.
            - Do NOT compare a categorical variable to a numeric `VALUE` node, and do NOT compare a numeric variable to a `CATEGORICAL_VALUE_NODE`.
            
        After writing your reasoning, you WILL generate the JSON object.


        """

    with user():
        llm += f"""
        **Document:**
        ---
        {document}
        ---

        **Query:**
        
        Based on the document, construct the computation graph for the following request:
        
        "{query}"

        ---

        **Initiate Process:**

        You will now follow the reasoning guidelines step-by-step before generating the final JSON.\n
        Step1. Analyze Query Parameters:\n

        {thought1}
        """

    # with assistant():
    #     # llm += "I will now follow the reasoning guidelines step-by-step before generating the final JSON.\n"
    #     # llm += "Step1. Analyze Query Parameters:\n"
    #     llm += guidance.json(
    #         name="thought1", 
    #         schema=ReasoningSchemaStep1, 
    #         max_tokens=1000)
    
    # with user():
    #     llm += "Step2. Identify All Relevant Information:\n"

    # with assistant():
    #     # llm += "Step2. Identify All Relevant Information:\n"
    #     llm += guidance.json(
    #         name="thought2", 
    #         schema=ReasoningSchemaStep2, 
    #         max_tokens=1000)

    # with user():
    #     llm += "Step3. Synthesize Plan:\n"

    # with assistant():
    #     # llm += "Step3. Synthesize Plan:\n"
    #     llm += guidance.json(
    #         name="thought3", 
    #         schema=ReasoningSchemaStep3, 
    #         max_tokens=1000)

    # with user():
    #     llm += "Step4. Rethink and Finalize Approach:\n"

    # with assistant():
    #     # llm += "Step4. Rethink and Finalize Approach:\n"
    #     llm += guidance.json(
    #         name="thought4", 
    #         schema=ReasoningSchemaStep4, 
    #         max_tokens=1000)

    # with user():
    #     llm += "Step5. Generate the JSON object:\n"
    #     # After thinking, it generates the JSON.
    #     # llm += "\n\nFinal JSON object:\n"
        
    with assistant():
        llm += gen_json(
            name="result_graph", 
            schema=output_schema,
            max_tokens=8000 
        )
        
    return llm

In [24]:

class ComputationGraphBuilder:
    """
    Orchestrates the creation of a computation graph by preparing dynamic
    constraints and prompting the LLM.
    """
    
    def __init__(self, model):
        """
        Initializes the builder with a guidance model.
        """
        self.model = model
        # Set the default LLM for all guidance programs
        # guidance.llm = self.model
    @timing_decorator
    def build(self, document_content: str, query: str, charge_category: str, charge_category_variables: dict, all_variables: dict) -> dict:
        """
        Generates a computation graph for a given query and document.

        Args:
            document_content: The text containing the rules.
            query: A natural language question about what to calculate.
            charge_category: The specific charge context used to filter variables.

        Returns:
            A dictionary representing the computation graph or an error.
        """
        print(f"--- Building graph for charge category: '{charge_category}' ---")
        
        # 1. Dynamically create the filtered Enum for this specific task
        try:
            Var = create_dynamic_variable_enum(charge_category, charge_category_variables)
        except ValueError as e:
            print(f"Error: {e}")
            return {"error": str(e)}

        def _var_type(v):
            try:
                return "CATEGORICAL" if v.data_type is str else "NUMERIC"
            except Exception:
                return "UNKNOWN"
        # 3. Create a formatted prompt string of allowed variables for the LLM
        allowed_variables = [el.value for el in list(Var)]
        allowed_variables_prompt = "\n".join(
            [
                f"- **{v.name}** | type: {_var_type(v)} | unit: {v.unit} — {v.description}"
                for name, v in all_variables.items() if name in allowed_variables
            ]
        )

        allowed_variables_t1_struct = "\n".join(
            [
                f"""variable name={v.name}, \n type_of_variable={_var_type(v)}, \n status=SYMBOLIC, \n description={v.description}"""
                for name, v in all_variables.items() if name in allowed_variables
            ]
        )
        # {ParameterDetail(name=v.name, status="SYMBOLIC", type_of_variable=_var_type(v)) for name, v in all_variables.items() if name in allowed_variables}
        

        try:
            # 4. Execute the guidance program with all dynamic components
            result_lm = self.model + create_graph_with_cot(
                allowed_variables_prompt=allowed_variables_prompt,
                thought1 = allowed_variables_t1_struct,
                document=document_content,
                query=query,
                output_schema=Node
            )
            
            
            # print("\nSuccessfully generated graph:")
            # # Use model_dump_json for Pydantic v2
            # print(pydantic_graph.model_dump_json(indent=2)) 
            return result_lm
            
        except Exception as e:
            print(f"\nAn error occurred while building the graph for '{query}': {e}")
            return {"error": str(e)}

In [25]:
import sympy
import re, keyword
from sympy import (
    Symbol, Piecewise, sympify, Add, Mul, Pow,
    Min, Max, floor, ceiling, Rational, Mod
)

def _sanitize_symbol_name(val: object) -> str:
    # name = re.sub(r"\W+", "_", str(val)).strip("_")
    name = re.sub(r"[^A-Za-z0-9]+", "", str(val))
    if not name:
        name = "_VAL"
    # Must start with letter or underscore
    if not re.match(r"[A-Za-z_]\w*$", name):
        name = "_" + name
    # Avoid reserved/boolean-like names
    if keyword.iskeyword(name) or name in {"None", "True", "False", "true", "false", "none"}:
        name = "_" + name
    return name.lower()

def compose_expression(node: dict):
    """
    Recursively parses a JSON graph into a SymPy expression,
    preventing automatic simplification.

    Args:
        node: A dictionary representing a node in the computation graph.

    Returns:
        A non-evaluated sympy expression representing the computation.
    """

    node_type = node.get('type')

    if node_type == "VALUE":
        return sympify(node['value'])
    
    elif node_type == "CATEGORICAL_VALUE":
        
        sanitized = _sanitize_symbol_name(node['value'])
        return Symbol(sanitized)
        # return sympify(sanitized)

    elif node_type == "VARIABLE":
        return Symbol(node['name'])

    elif node_type == "BINARY_OPERATION":
        left = compose_expression(node['left'])
        right = compose_expression(node['right'])
        operator = node['operator']
        
        # Use class constructors with evaluate=False to prevent simplification
        if operator == "ADD":
            return Add(left, right, evaluate=False)
        elif operator == "MULTIPLY":
            return Mul(left, right, evaluate=False)
        elif operator == "SUBTRACT":
            return Add(left, Mul(-1, right, evaluate=False), evaluate=False)
        elif operator == "FLOOR_DIVIDE":
            q = Mul(left, Pow(right, -1, evaluate=False), evaluate=False)
            return floor(q, evaluate=False)
        elif operator == "MOD":
            return Mod(left, right, evaluate=False)
        elif operator == "POWER":
            return Pow(left, right, evaluate=False)
        elif operator == "ROOT":
            inv = Pow(right, -1, evaluate=False)
            return Pow(left, inv, evaluate=False)
        elif operator == "SQRT":
            return Pow(left, Rational(1, 2), evaluate=False)
        elif operator == "FLOOR":
            return floor(left, evaluate=False)
        elif operator == "CEILING":
            return ceiling(left, evaluate=False)
        elif operator == "MIN":
            return Min(left, right, evaluate=False)
        elif operator == "MAX":
            return Max(left, right, evaluate=False)
        elif operator == "DIVIDE":
            # Division (a/b) is represented as a * (b**-1)
            power = Pow(right, -1, evaluate=False)
            return Mul(left, power, evaluate=False)
        else:
            raise ValueError(f"Unsupported binary operator: {operator}")

    elif node_type == "COMPARISON":
        left = compose_expression(node['left'])
        right = compose_expression(node['right'])
        operator = node['operator']

        if operator == "GREATER_THAN":
            return left > right
        elif operator == "LESS_THAN":
            return left < right
        elif operator == "EQUAL_TO":
            return sympy.Eq(left, right, evaluate=False)
        elif operator == "NOT_EQUAL_TO":
            return sympy.Ne(left, right, evaluate=False)
        elif operator == "LESS_THAN_OR_EQUAL_TO":
            return sympy.Le(left, right, evaluate=False)
        elif operator == "GREATER_THAN_OR_EQUAL_TO":
            return sympy.Ge(left, right, evaluate=False)
        else:
            raise ValueError(f"Unsupported comparison operator: {operator}")

    elif node_type == "CONDITIONAL":
        condition = compose_expression(node['condition'])
        if_true_expr = compose_expression(node['if_true'])
        if_false_expr = compose_expression(node['if_false'])
        
        return Piecewise((if_true_expr, condition), (if_false_expr, True))

    else:
        print(node)
        raise ValueError(f"Unknown node type: {node_type}")

In [26]:
import json
import traceback


def create_computation_graph(model, query, charge_category, markdown_content, charge_category_variables, all_variables):
    graph_builder = ComputationGraphBuilder(model=model)

    start_time = time.perf_counter()
    llm_structured_response = graph_builder.build(
        document_content=markdown_content,
        query=query,
        charge_category=charge_category,
        charge_category_variables=charge_category_variables,
        all_variables=all_variables
    )
    end_time = time.perf_counter()
    build_time = end_time - start_time

    return llm_structured_response, build_time
    

# Evals 

## Pre-Processing

In [27]:
from json import JSONDecodeError

def generate(path):
    computed_records = {}
    with open(path, "r", encoding="utf-8") as f:
        records = [json.loads(line) for line in f if line.strip()]
    
    for record_id, record in enumerate(records):
        if record_id >= 5:
            continue
        content, charges = record
        markdown_content = content
        charge_category_variables = {k: v['variables_used'] for k, v in charges.items()}
        computed_records[str(record_id)] = {}
        import builtins
        def to_built_in_type(type_str: str):
            t = getattr(builtins, type_str, None)
            if isinstance(t, type):
                return t
            raise ValueError(f"{type_str} is not a valid built-in type")

        charge_variables = {}
        all_variables = {}
        for charge, variables in charge_category_variables.items():
            charge_variables[charge] = list(variables.keys())
            for var, details in variables.items():
                all_variables[var] = DomainVariable(name=var,
                    description=details['description'],
                    unit=details['unit'],
                    data_type=to_built_in_type(details['dtype']))

        for charge_category in list(charge_variables.keys()):
            query = f"Calculate the total {charge_category}. Charge description - {charges[charge_category]['charge_description']}"    
            print(query, charge_category, charge_variables, all_variables)
            result, execution_time = create_computation_graph(base_lm, query, charge_category, markdown_content, charge_variables, all_variables)
            
            try:
                payload = {charge_category: {
                    'query': query,
                    # 'thought2':json.loads(result.get("thought2")),
                    # 'thought3':json.loads(result.get("thought3")),
                    # 'thought4':json.loads(result.get("thought4")),
                    'reasoning': result.get('reasoning'),
                    'computation_graph':json.loads(result.get("result_graph")),
                    'execution_time': execution_time,
                    'status': 'SUCCESS'
                }
                }
                computed_records[str(record_id)][charge_category] = payload
            except JSONDecodeError:
                payload = {charge_category: {
                    'query': query,
                    # 'thought2':result.get("thought2", None),
                    # 'thought3':result.get("thought3", None),
                    # 'thought4':result.get("thought4", None),
                    'reasoning': result.get('reasoning', None),
                    'computation_graph':result.get("result_graph", None),
                    'execution_time': execution_time,
                    'status': 'ERROR'
                }
                }
            except Exception as e:
                payload = {charge_category: {
                    'query': query,
                    # 'thought2':None,
                    # 'thought3':None,
                    # 'thought4':None,
                    'reasoning': None,
                    'computation_graph':None,
                    'execution_time': execution_time,
                    'status': f'ERROR - {e}'
                }
                }

            
    return computed_records

In [28]:
from sympy import lambdify, Symbol

def evaluate(computation_graph, charge_variable, ground_truth_code):
    symbolic_expr = compose_expression(computation_graph)
    
    charge, variables = charge_variable
    symbolic_vars = tuple([Symbol(var) for var in variables])
    
    symbolic_func = lambdify(symbolic_vars, symbolic_expr, 'sympy')

    ns = {}
    exec(ground_truth_code, ns)                 # defines compute_charge in ns
    ground_truth_func = ns['compute_charge']

    return symbolic_func, ground_truth_func


In [ ]:
# path = "/gpfs01/home/ppxac9/MLiS-Placement-Thesis/LLM_generated_data/synthetic_dataset/datapoints_8B.jsonl"
# # path = "/gpfs01/home/ppxac9/MLiS-Placement-Thesis/LLM_generated_data/synthetic_dataset/datapoints_70B_4bit.jsonl"
# result = generate(path)

Calculate the total compute_vm_fee. Charge description - Charge for running virtual machines based on vCPU, memory, OS, and commitment. compute_vm_fee {'compute_vm_fee': ['os_type', 'usage_hours'], 'gpu_compute_fee': ['gpu_type'], 'managed_disk_fee': ['managed_disk_count'], 'data_egress_fee': ['region'], 'data_ingress_fee': ['region', 'data_ingress_gb'], 'storage_transaction_fee': ['redundancy', 'storage_transactions_millions'], 'public_ip_fee': ['region', 'public_ip_count'], 'load_balancer_fee': ['load_balancer_rules'], 'aks_cluster_fee': ['region'], 'function_app_fee': ['region', 'function_exec_ms'], 'sql_database_fee': ['region', 'backup_retention_days'], 'snapshot_storage_fee': ['redundancy'], 'app_service_fee': ['service_tier', 'app_service_instances']} {'os_type': DomainVariable(name='os_type', description='Operating system for the compute workload', unit='os', data_type=<class 'str'>), 'usage_hours': DomainVariable(name='usage_hours', description='Total runtime in hours over the

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

Execution time for 'build': 543.3702 seconds
Calculate the total gpu_compute_fee. Charge description - Charge for GPU-enabled compute usage. gpu_compute_fee {'compute_vm_fee': ['os_type', 'usage_hours'], 'gpu_compute_fee': ['gpu_type'], 'managed_disk_fee': ['managed_disk_count'], 'data_egress_fee': ['region'], 'data_ingress_fee': ['region', 'data_ingress_gb'], 'storage_transaction_fee': ['redundancy', 'storage_transactions_millions'], 'public_ip_fee': ['region', 'public_ip_count'], 'load_balancer_fee': ['load_balancer_rules'], 'aks_cluster_fee': ['region'], 'function_app_fee': ['region', 'function_exec_ms'], 'sql_database_fee': ['region', 'backup_retention_days'], 'snapshot_storage_fee': ['redundancy'], 'app_service_fee': ['service_tier', 'app_service_instances']} {'os_type': DomainVariable(name='os_type', description='Operating system for the compute workload', unit='os', data_type=<class 'str'>), 'usage_hours': DomainVariable(name='usage_hours', description='Total runtime in hours ov

In [43]:
import pickle
# with open('result_8b_30bthink_3.pkl', 'wb') as f:
#     pickle.dump(, f)

with open('result_8b_30bthink_3.pkl', 'rb') as f:
    result = pickle.load(f)


In [28]:
from json import JSONDecodeError

def generate_missing(path, _missing_charge, _doc_id):
    computed_records = {}
    with open(path, "r", encoding="utf-8") as f:
        records = [json.loads(line) for line in f if line.strip()]
    
    for record_id, record in enumerate(records):
        if record_id != _doc_id:
            continue
        content, charges = record
        markdown_content = content
        charge_category_variables = {k: v['variables_used'] for k, v in charges.items()}
        computed_records[str(record_id)] = {}
        import builtins
        def to_built_in_type(type_str: str):
            t = getattr(builtins, type_str, None)
            if isinstance(t, type):
                return t
            raise ValueError(f"{type_str} is not a valid built-in type")

        charge_variables = {}
        all_variables = {}
        for charge, variables in charge_category_variables.items():
            charge_variables[charge] = list(variables.keys())
            for var, details in variables.items():
                all_variables[var] = DomainVariable(name=var,
                    description=details['description'],
                    unit=details['unit'],
                    data_type=to_built_in_type(details['dtype']))

        for charge_category in list(charge_variables.keys()):
            if charge_category != _missing_charge:
                continue
            query = f"Calculate the total {charge_category}. Charge description - {charges[charge_category]['charge_description']}"    
            print(query, charge_category, charge_variables, all_variables)
            result, execution_time = create_computation_graph(base_lm, query, charge_category, markdown_content, charge_variables, all_variables)
            
            try:
                payload = {charge_category: {
                    'query': query,
                    # 'thought2':json.loads(result.get("thought2")),
                    # 'thought3':json.loads(result.get("thought3")),
                    # 'thought4':json.loads(result.get("thought4")),
                    'reasoning': result.get('reasoning'),
                    'computation_graph':json.loads(result.get("result_graph")),
                    'execution_time': execution_time,
                    'status': 'SUCCESS'
                }
                }
                computed_records[str(record_id)][charge_category] = payload
            except JSONDecodeError:
                payload = {charge_category: {
                    'query': query,
                    # 'thought2':result.get("thought2", None),
                    # 'thought3':result.get("thought3", None),
                    # 'thought4':result.get("thought4", None),
                    'reasoning': result.get('reasoning', None),
                    'computation_graph':result.get("result_graph", None),
                    'execution_time': execution_time,
                    'status': 'ERROR'
                }
                }
                computed_records[str(record_id)][charge_category] = payload
            except Exception as e:
                payload = {charge_category: {
                    'query': query,
                    # 'thought2':None,
                    # 'thought3':None,
                    # 'thought4':None,
                    'reasoning': None,
                    'computation_graph':None,
                    'execution_time': execution_time,
                    'status': f'ERROR - {e}'
                }
                }
                computed_records[str(record_id)][charge_category] = payload

            
    return computed_records

# Testing

In [29]:

def get_symbolic_groundtruth_functions(path, result_path):
    master_function_list = {}
    error_list = {}
    missing_list = []
    with open(path, "r", encoding="utf-8") as f:
            records = [json.loads(line) for line in f if line.strip()]
    with open(result_path, 'rb') as f:
        result = pickle.load(f)
    for record_id, record in enumerate(records):
        content, charges = record
        markdown_content = content
        charge_category_variables = {k: v['variables_used'] for k, v in charges.items()}
        master_function_list[record_id] = {}
        error_list[record_id] = {}
        import builtins
        def to_built_in_type(type_str: str):
            t = getattr(builtins, type_str, None)
            if isinstance(t, type):
                return t
            raise ValueError(f"{type_str} is not a valid built-in type")

        charge_variables = {}
        all_variables = {}
        for charge, variables in charge_category_variables.items():
            charge_variables[charge] = list(variables.keys())
            for var, details in variables.items():
                all_variables[var] = DomainVariable(name=var,
                    description=details['description'],
                    unit=details['unit'],
                    data_type=to_built_in_type(details['dtype']))
            try:
                graph = result[str(record_id)][charge][charge]['computation_graph']
                code_str = charges[charge]['code']
            except KeyError as e:
                print(record_id, charge, result[str(record_id)].keys(), )
                missing_list.append((record_id, charge))
            try:
                symbolic_func, ground_truth_func = evaluate(graph, (charge, charge_variables[charge]), code_str)
                master_function_list[record_id][charge] = (symbolic_func, ground_truth_func, graph)
            except Exception  as e:
                error_list[record_id][charge] = [graph, e]
            
    return master_function_list, error_list, missing_list

In [30]:
path = "/gpfs01/home/ppxac9/MLiS-Placement-Thesis/LLM_generated_data/synthetic_dataset/datapoints_8B.jsonl"
result_path='result_8b_30bthink_3.pkl'
functions, errors, missing_list = get_symbolic_groundtruth_functions(path, result_path)

{'operator': 'EQUAL_TO', 'left': {'type': 'VARIABLE', 'name': 'os_type', 'description': 'Operating system for the compute workload', 'unit': 'CENTIMETERS'}, 'right': {'type': 'CATEGORICAL_VALUE', 'value': 'Linux', 'description': 'Linux operating system'}}
0 storage_transaction_fee dict_keys(['gpu_compute_fee', 'managed_disk_fee', 'data_egress_fee', 'data_ingress_fee', 'load_balancer_fee', 'aks_cluster_fee', 'compute_vm_fee'])
0 public_ip_fee dict_keys(['gpu_compute_fee', 'managed_disk_fee', 'data_egress_fee', 'data_ingress_fee', 'load_balancer_fee', 'aks_cluster_fee', 'compute_vm_fee'])
0 function_app_fee dict_keys(['gpu_compute_fee', 'managed_disk_fee', 'data_egress_fee', 'data_ingress_fee', 'load_balancer_fee', 'aks_cluster_fee', 'compute_vm_fee'])
0 sql_database_fee dict_keys(['gpu_compute_fee', 'managed_disk_fee', 'data_egress_fee', 'data_ingress_fee', 'load_balancer_fee', 'aks_cluster_fee', 'compute_vm_fee'])
0 snapshot_storage_fee dict_keys(['gpu_compute_fee', 'managed_disk_fee',

In [ ]:
# with open(path, "r", encoding="utf-8") as f:
#     records = [json.loads(line) for line in f if line.strip()]


In [ ]:
with open(path, "r", encoding="utf-8") as f:
    records = [json.loads(line) for line in f if line.strip()]

master_evals = {}
count_c = 0
count_w = 0
for record_id, record in enumerate(records):
    if record_id >= 5:
        continue
    content, charges = record
    charge_category_variables = {k: v['variables_used'] for k, v in charges.items()}
    master_evals[record_id] = {}
    charge_variables = {}
    all_variables = {}
    for charge, variables in charge_category_variables.items():
        master_evals[record_id][charge] = []
        # print(record_id, charge)
        if charge not in functions[record_id].keys():
            continue
        generated_func, ground_truth_func, graph = functions[record_id][charge]
        code_str = charges[charge]['code']
        tests = create_tests(code_str)
        ground_truth_func_evals = {}
        generated_func_test_evals = {}
        for condition_no, test in enumerate(tests):
            ground_truth_func_evals[condition_no] = [coerce_literals(test) for test in cartesian_combine_lists(*test)]
            generated_func_test_evals[condition_no] = symbolize_records([coerce_literals(test) for test in cartesian_combine_lists(*test)])

        ground_truth_computed_result = []
        symbolic_func_computed_result = []
        for i, condition in ground_truth_func_evals.items():
            for test in condition:
                try:
                    result = ground_truth_func(**test)
                except Exception as e:
                    count_w += 1
                    print('!!!', 'ground truth', e)
                    continue
                # if charge == 'gpu_compute_fee' and record_id == 3:
                #     print(test, result)
                ground_truth_computed_result.append(result)

        for i, condition in generated_func_test_evals.items():
            for test in condition:
                try:
                    result = generated_func(**test)
                except Exception as e:
                    count_w += 1
                    # print('!!!', e)
                    continue
                # if charge == 'gpu_compute_fee' and record_id == 3:
                #     print(">",test, result)
                symbolic_func_computed_result.append(result)
        # invalid documents from the 8B corpus
        exception_list = [(0, 'compute_vm_fee'), 
                        (0, 'data_ingress_fee'), 
                        (1, 'storage_transaction_fee'),
                        (1, 'data_ingress_fee'),
                        (1, 'compute_vm_fee'),
                        (1, 'gpu_compute_fee'),
                        (1, 'public_ip_fee'),
                        (2, 'public_ip_fee'),
                        (2, 'function_app_fee'),
                        (4, 'data_ingress_fee'),
                        (2, 'data_egress_fee'),
                        (2, 'aks_cluster_fee'),
                        (4, 'function_app_fee')]
        # invalid documents from the 70B corpus
        # exception_list_70b = [
        #     (0, 'compute_vm_fee'),
        #     (0, 'gpu_compute_fee'),
        #     (0, 'public_ip_fee'),
        #     (0, 'load_balancer_fee'),
        #     (1, 'snapshot_storage_fee'),
        #     (1, 'app_service_fee'),
        #     (3, 'data_ingress_fee'),
        #     (3, 'load_balancer_fee'),
        # ]
        for symr, gtr in zip(symbolic_func_computed_result, ground_truth_computed_result):
            if isinstance(symr, float):
                master_evals[record_id][charge].append([symr, gtr, round(symr, 6) == round(gtr, 6)])
            else:
                master_evals[record_id][charge].append([symr, gtr, symr == gtr])
            if (record_id, charge) not in exception_list:    
                # print(record_id, charge)
                # print(record_id, charge)
                if isinstance(symr, float):
                    if round(symr, 6) != round(gtr, 6):
                        print(record_id, charge, symr, gtr)
                        count_w += 1
                elif symr != gtr:
                    print(record_id, charge, symr, gtr)
                    count_w += 1

                # if isinstance(symr, float):
                #     if round(symr, 6) == round(gtr, 6):
                #         print(record_id, charge, symr, gtr)
                #         count_c += 1
                # elif symr == gtr:
                #     print(record_id, charge, symr, gtr)
                #     count_c += 1


0 data_ingress_fee 71866.0 143732.0
0 data_ingress_fee 93868.0 281604.0
0 data_ingress_fee 59480.3 118960.6
0 data_ingress_fee 103600.20000000001 310800.60000000003
0 data_ingress_fee 82426.8 164853.6
0 data_ingress_fee 110142.0 330426.0
0 data_ingress_fee 75699.0 151398.0
0 data_ingress_fee 115759.8 347279.4
0 data_ingress_fee 95390.4 190780.8
0 data_ingress_fee 133929.6 401788.8
0 data_ingress_fee 96462.0 192924.0
0 data_ingress_fee 141939.0 425817.0
!!! ground truth compute_charge() got an unexpected keyword argument 'redundancy'
!!! ground truth compute_charge() got an unexpected keyword argument 'redundancy'
!!! ground truth compute_charge() got an unexpected keyword argument 'redundancy'
!!! ground truth compute_charge() got an unexpected keyword argument 'redundancy'
!!! ground truth compute_charge() got an unexpected keyword argument 'redundancy'
!!! ground truth compute_charge() got an unexpected keyword argument 'redundancy'
!!! ground truth compute_charge() got an unexpected

In [34]:
import ast

def extract_if_conditions(code_str):
    """
    Extracts conditions from if-else statements in a Python function string.
    
    Args:
    code_str (str): The source code of the function as a string.
    
    Returns:
    list[str]: A list of condition strings (e.g., "x > 0" or "else").
    """
    try:
        tree = ast.parse(code_str)
    except SyntaxError as e:
        raise ValueError(f"Invalid Python code: {e}")
    
    class IfConditionVisitor(ast.NodeVisitor):
        def __init__(self):
            self.conditions = []
        
        def visit_If(self, node):
            current = node
            while True:
                self.conditions.append(ast.unparse(current.test).strip())
                
                # Visit statements in the body for nested structures
                for stmt in current.body:
                    self.visit(stmt)
                
                if current.orelse:
                    if len(current.orelse) == 1 and isinstance(current.orelse[0], ast.If):
                        # Handle elif by continuing the loop
                        current = current.orelse[0]
                        continue
                    else:
                        # Handle else
                        self.conditions.append("else")
                        # Visit statements in the else body
                        for stmt in current.orelse:
                            self.visit(stmt)
                
                # End of chain
                break
    
    visitor = IfConditionVisitor()
    visitor.visit(tree)
    
    return visitor.conditions

In [35]:
import re

def create_tests(ground_truth_code_string):
    binray_conditionals = ['and']
    numeric_conditionals = ['>=', "<=", "==", '<', '>']
    ops_pat = '|'.join(map(re.escape, numeric_conditionals))
    test_cases = []

    if_conditions = extract_if_conditions(ground_truth_code_string)
    
    for condition in if_conditions:
        condition_level_test = []
        expressions = [t.strip() for t in re.split(rf'\s*({'|'.join(binray_conditionals)})\b\s*', condition, flags=re.IGNORECASE)]
    
        for expression in expressions:
            
            if expression == 'else':
                continue    
            if expression == 'and':
                continue
            expression_components = [t.strip() for t in re.split(rf'\s*({ops_pat})\s*', expression)]
            # print(expression_components)
            test = parse_expression(expression_components)
            condition_level_test.append(test)
        if condition_level_test != []:
            test_cases.append(condition_level_test)
    return test_cases

In [36]:
import random
import math

def sample_values(tokens, n=5, seed=None, domain=None, integer=False):
    """
    tokens:
      - 5 tokens: [L, op1, var, op2, R]
      - 3 tokens: [var, op, value] OR [value, op, var]
    operators: <, <=, >, >=, ==
    Returns n samples satisfying the constraint. Floats by default; ints if integer=True.
    For single-sided constraints (3 tokens), pass domain=(lo, hi) to bound the other side.
    """
    if seed is not None:
        random.seed(seed)

    def is_number(s):
        try:
            float(s)
            return True
        except Exception:
            return False

    # Collect bounds as (value, inclusive) or None
    lower = None   # (lo_val, lo_inclusive)
    upper = None   # (hi_val, hi_inclusive)
    eq_value = None

    if len(tokens) == 5:
        L, op1, var, op2, R = tokens
        a = float(L)
        b = float(R)

        # Left comparison: L op1 var  -> constraint on var
        if op1 == '<':
            # L < var  => var > L  (lower, open)
            lower = (a, False)
        elif op1 == '<=':
            # L <= var => var >= L (lower, closed)
            lower = (a, True)
        elif op1 == '>':
            # L > var  => var < L  (upper, open)
            upper = (a, False)
        elif op1 == '>=':
            # L >= var => var <= L (upper, closed)
            upper = (a, True)
        elif op1 == '==':
            eq_value = a
        else:
            raise ValueError(f"Unsupported operator: {op1}")

        # Right comparison: var op2 R -> constraint on var
        if op2 == '<':
            # var < R  (upper, open)
            upper = (b, False) if upper is None else min(upper, (b, False), key=lambda t:(t[0], t[1]==False))
        elif op2 == '<=':
            # var <= R (upper, closed)
            upper = (b, True) if upper is None else min(upper, (b, True), key=lambda t:(t[0], t[1]==False))
        elif op2 == '>':
            # var > R  (lower, open)
            lower = (b, False) if lower is None else max(lower, (b, False), key=lambda t:(t[0], t[1]==True))
        elif op2 == '>=':
            # var >= R (lower, closed)
            lower = (b, True) if lower is None else max(lower, (b, True), key=lambda t:(t[0], t[1]==True))
        elif op2 == '==':
            if eq_value is not None and abs(eq_value - b) > 0:
                # contradictory equals
                return []
            eq_value = b
        else:
            raise ValueError(f"Unsupported operator: {op2}")

    elif len(tokens) == 3:
        x, op, y = tokens

        # Normalize to "var ? value" direction
        if is_number(x) and not is_number(y):
            # pattern: value op var  -> flip to var flip(op) value
            val = float(x)
            var = y
            flip = {'<': '>', '<=': '>=', '>': '<', '>=': '<=', '==': '=='}
            op = flip.get(op)
            if op is None:
                raise ValueError("Unsupported operator")
        elif not is_number(x) and is_number(y):
            # pattern: var op value (already normalized)
            var = x
            val = float(y)
        else:
            raise ValueError("Ambiguous tokens: need exactly one numeric and one variable")

        # Single-sided constraint -> set a bound
        if op == '<':
            upper = (val, False)
        elif op == '<=':
            upper = (val, True)
        elif op == '>':
            lower = (val, False)
        elif op == '>=':
            lower = (val, True)
        elif op == '==':
            eq_value = val
        else:
            raise ValueError(f"Unsupported operator: {op}")
    else:
        raise ValueError("Unexpected number of tokens")

    # Handle equality (must also satisfy other bounds if present)
    if eq_value is not None:
        v = float(eq_value)
        if lower is not None:
            lo, lo_inc = lower
            if not (v > lo or (lo_inc and v == lo)):
                return []
        if upper is not None:
            hi, hi_inc = upper
            if not (v < hi or (hi_inc and v == hi)):
                return []
        if integer:
            if abs(v - round(v)) > 1e-12:
                return []  # equality to a non-integer but integer requested
            v = int(round(v))
        return [v for _ in range(n)]

    # Fill missing side from domain (if needed)
    if lower is None or upper is None:
        if domain is None:
            raise ValueError("Single-sided constraint: please provide domain=(lo, hi)")
        dom_lo, dom_hi = map(float, domain)
        if lower is None:
            lower = (dom_lo, True)
        if upper is None:
            upper = (dom_hi, True)

    lo, lo_inc = lower
    hi, hi_inc = upper

    # Empty interval check
    if hi < lo or (hi == lo and (not lo_inc or not hi_inc)):
        return []

    if integer:
        # Convert to integer inclusive bounds
        lo_i = (math.floor(lo) + 1) if not lo_inc else math.ceil(lo)
        hi_i = (math.ceil(hi) - 1) if not hi_inc else math.floor(hi)
        if lo_i > hi_i:
            return []
        return [random.randint(lo_i, hi_i) for _ in range(n)]

    # Float sampling; nudge for open bounds
    eps = 1e-12 * max(1.0, abs(lo), abs(hi))
    lo_f = lo if lo_inc else lo + eps
    hi_f = hi if hi_inc else hi - eps
    if hi_f < lo_f:
        return []

    return [random.uniform(lo_f, hi_f) for _ in range(n)]

In [37]:
import itertools

def cartesian_combine_lists(*lists_of_dicts):
    """
    Each arg is a list[dict]. Returns list[dict] for the Cartesian product.
    If any list is empty -> returns [].
    """
    out = []
    for prod in itertools.product(*lists_of_dicts):
        d = {}
        for part in prod:
            d.update(part)  # later lists override earlier on key conflicts
        out.append(d)
    return out

In [38]:
import ast

def coerce_literals(rec):
    out = {}
    for k, v in rec.items():
        if isinstance(v, str):
            try:
                v = ast.literal_eval(v)  # "'linux'" -> "linux", "598.4" -> 598.4
            except Exception:
                pass
        out[k] = v
    return out

# results = [compute_charge(**coerce_literals(rec)) for rec in records]

In [39]:
from sympy import Symbol

def symbolize_records(records, only_keys=None, skip_numeric_like=True, in_place=False):
    """
    records: list of dicts
    only_keys: set/iterable of keys to convert; if None => convert all str values
    skip_numeric_like: don't convert strings that parse as numbers (e.g. "123.4")
    in_place: mutate input list if True; otherwise return a new list
    """
    def is_numlike(s):
        try:
            float(s)
            return True
        except Exception:
            return False

    target = records if in_place else [dict(r) for r in records]
    for rec in target:
        for k, v in list(rec.items()):
            if isinstance(v, str) and (only_keys is None or k in only_keys):
                if skip_numeric_like and is_numlike(v):
                    continue
                rec[k] = Symbol(_sanitize_symbol_name(v))
    return target

In [40]:
from typing import List

def parse_expression(expression: List):
    if '==' in expression:
        return [{expression[0]: expression[2]}]
    else:
        values = sample_values(expression, n=1, integer=True)

        if len(expression) == 5:
            cases = [{expression[2]: v} for v in values]
            return cases
        elif len(expression) == 3:
            cases = [{expression[0]: v} for v in values]
            return cases
        else:
            raise('Unexpected Expression Length')